# Zipline Installation: Bundle Ingest & Algorithm Test

## Imports and Settings

We don't need much here since the imports all take place in the cell where we formulate the algorithm.

In [9]:
%matplotlib inline

## Load Zipline extension

In [10]:
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


## Ingest Data

In [11]:
!zipline ingest -b quandl

Merging daily equity files:


[2024-06-23T09:45:38+1200-INFO][zipline.data.bundles.core]
 Ingesting quandl
[2024-06-23T09:45:38+1200-INFO][zipline.data.bundles.quandl]
[2024-06-23T09:47:26+1200-INFO][zipline.data.bundles.quandl]
 Parsing raw data.
[2024-06-23T09:47:44+1200-INFO][zipline.data.bundles.quandl]
 Generating asset metadata.
C:\tools\miniforge-pypy3\envs\liveproject\lib\site-packages\zipline\data\bundles\quandl.py:106: FutureWarning: The provided callable <function min at 0x000001547DE8CD30> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  data = data.groupby(by="symbol").agg({"date": [np.min, np.max]})
C:\tools\miniforge-pypy3\envs\liveproject\lib\site-packages\zipline\data\bundles\quandl.py:106: FutureWarning: The provided callable <function max at 0x000001547DE8CC10> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To

## Run a simple test algorithm

In [12]:
%%zipline --start 2014-1-1 --end 2018-1-1 -o dma.pickle


from zipline.api import order_target, record, symbol
import matplotlib.pyplot as plt

def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=100, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=300, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(AAPL=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)


def analyze(context, perf):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value in $')

    ax2 = fig.add_subplot(212)
    perf['AAPL'].plot(ax=ax2)
    perf[['short_mavg', 'long_mavg']].plot(ax=ax2)

    perf_trans = perf.loc[[t != [] for t in perf.transactions]]
    buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.loc[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax2.plot(buys.index, perf.short_mavg.loc[buys.index],
             '^', markersize=10, color='m')
    ax2.plot(sells.index, perf.short_mavg.loc[sells.index],
             'v', markersize=10, color='k')
    ax2.set_ylabel('price in $')
    plt.legend(loc=0)
    plt.show()
    

No benchmark configured. Assuming algorithm calls set_benchmark.
Pass --benchmark-sid, --benchmark-symbol, or --benchmark-file to set a source of benchmark returns.
Pass --no-benchmark to use a dummy benchmark of zero returns.


DateOutOfBounds: Parameter `end` receieved as '2025-06-23 00:00:00' although cannot be later than the last session of calendar 'XNYS' ('2025-06-20 00:00:00').